## Encontrar SPAN

In [ ]:
#IMPORTAR LIBRERIAS
import pandas as pd
import spacy
from spacy.lang.es import Spanish
from spacy.pipeline import EntityRuler
from tqdm import tqdm, trange
import os
import re

In [ ]:
def preprocess(text):
    nlp = Spanish()
    result= []
    for i in trange(0,len(text)):
        text[i]=str(text[i])
        text[i]=re.sub(u'^(?<![a-z A-Z])',' ',text[i])
        text[i]=text[i].lower()
        text[i]=text[i].replace('.',' ')
        text[i]=text[i].replace('-',' ')
        a,b = 'áéíóúü','aeiouu'
        trans = str.maketrans(a,b)
        text[i]=text[i].translate(trans)
        text[i]=text[i].strip()
        doc=nlp(text[i])
        t=[token.text for token in doc if not token.is_punct]
        result.append(' '.join(list(t)))
    return result

In [5]:
#CARGAR EL DATASET POR HOJA A PROCESAR
db=pd.read_excel('Data.xlsx', delimiter=';',encoding='latin-1')
#cuidar de nno tener valores NaN
values={'NaN':'Ninguna'}
db=db.fillna(value=values)
db=db.reset_index()

In [1]:
#Preprocesar el texto de la base de datos
db['Texto']=preprocess(db['Texto'])

In [ ]:
#Funcion encontrar proveedor
def find_SPAN(sentence,diccionario):#,sheet):
    #Esta funcion esta implementada para 2 posibles nombres de una sola entidad ejemplo:
    #Banco de Crédito de Perú: BCP, BCP-Banco de credito, banco de credito
    df=pd.read_excel(diccionario, delimiter=';',encoding='latin-1')#puedes tener varios sheets con diferentes diccionarios en una sola hoja excel
    #creardo diccionarios en la estructura que Spacy requiere
    df1=df[['label','pattern']]
    df2=df[['label','pattern.1']]
    df2=df2.rename(columns={'pattern.1':'pattern'}, inplace = False)
    #Crear los diccionarios #si los nombres del conjunto SPAN son mas de 2 es necesario tambien implementar 
    #mas patterns
    patterns1=df1.to_dict(orient='records')
    patterns2=df2.to_dict(orient='records')
    #patternsn=dfn.to_dict(orient='records')
    patterns=patterns1+patterns2
    #Crear el objeto NLP
    nlp = Spanish()
    ruler = EntityRuler(nlp)
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler)

    #Preprocesar la entrada
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans(a,b)
    sentence=sentence.lower().translate(trans)
    doc = nlp(sentence)
    #Devuelve la etiqueta real de la entidad
    if (sentence=='' or  sentence==' ' or sentence=='Por definir' or sentence=='por definir' or
       sentence=='no hay datos ' or sentence=='nan'):
        label=['No encontrado']#con esto evitaremos el rpoblema de comillas
    else: 
        label= list(set([ent.label_ for ent in doc.ents]))
    #label no establecida
    if label!=[]:
        label=label
    else: label=['No encontrado']#con esto evitaremos el problema de comillas
    return label[0]#agregando el[0]obtenemos solo el elemento entre comillas sin corchetes y solo un proveedor

In [ ]:
#APLICAR LA FUNCION find_SPAN
#Se generas una columna en el data set con los SPAN encontrados
db['SPAN']=db.apply(lambda x: find_SPAN(str(x['Texto']),'Diccionario.xlsx'),axis=1)

In [ ]:
db.to_excel('Data_span.xlsx',index=False)